In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!ls /content/drive/My\ Drive/Kaggle/data

test_out  train  validate


In [ ]:
!ls /content/drive/My\ Drive/Kaggle/data/train
!ls /content/drive/My\ Drive/Kaggle/data/validate
!ls /content/drive/My\ Drive/Kaggle/data/test_out

Adults	Teenagers  Toddlers
Adults	Teenagers  Toddlers
test  Test.csv


In [ ]:
!ls /content/drive/My\ Drive/Kaggle/data/validate/Adults

00000516.jpg
00000530.jpg
00000537.jpg
00000538.jpg
00000543.jpg
00000546.jpg
00000548.jpg
00000549.jpg
00000550.jpg
00000551.jpg
00000553.jpg
00000557.jpg
00000559.jpg
00000562.jpg
00000568.jpg
00000570.jpg
00000577.jpg
00000580.jpg
00000581.jpg
00000587.jpg
00000590.jpg
00000595.jpg
00000597.jpg
00000598.jpg
00000600.jpg
00000601.jpg
0010.jpg
0011.jpg
0012.jpg
0013.jpg
0014.jpg
0015.jpg
0016.jpg
0017.jpg
0018.jpg
0019.jpg
001.jpg
0020.jpg
0021.jpg
0022.jpg
0023.jpg
0024.jpg
0025.jpg
0026.jpg
0027.jpg
0028.jpg
0029.jpg
002.jpg
0030.jpg
0031.jpg
0032.jpg
0033.jpg
0034.jpg
0035.jpg
0036.jpg
0037.jpg
0038.jpg
0039.jpg
003.jpg
0040.jpg
0041.jpg
0042.jpg
0043.jpg
0044.jpg
0045.jpg
0046.jpg
004.jpg
005.jpg
006.jpg
007.jpg
008.jpg
009.jpg
5x5-grid-close-ups-smiling-260nw-216231307.jpg
active-adults-playing-beachvolley-happy-throwing-ball-sandy-summer-beach-near-blue-sea-87274071.jpg
addicted-woman-sharing-her-rehabilitation-260nw-1656777319.jpg
adult-group-celebrating-friend-winning-260nw-10

In [ ]:
!pip install tensorflow numpy scikit-learn matplotlib

In [ ]:
import os
import numpy as np
from sklearn.preprocessing import LabelEncoder
from collections import Counter
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications.resnet_v2 import ResNet50V2, preprocess_input
from tensorflow.keras.layers import Dense, Dropout, GlobalAveragePooling2D
from tensorflow.keras.models import Model
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau
from tensorflow.keras import regularizers
import matplotlib.pyplot as plt

In [ ]:
# Ensure reproducibility
tf.random.set_seed(42)
np.random.seed(42)

# Step 1: Set up directory paths (data already unzipped)
data_dir = "/content/drive/My Drive/Kaggle/data"
train_path = os.path.join(data_dir, "train")
validate_path = os.path.join(data_dir, "validate")
test_path = os.path.join(data_dir, "test_out")

# Verify directories exist
if not all(os.path.exists(p) for p in [train_path, validate_path, test_path]):
    raise FileNotFoundError("One or more data directories not found. Check train, validate, and test_out in /content/drive/My Drive/Kaggle/data")

In [ ]:
# Step 2: Skip validation set creation (already done by user)
print("Validation set assumed to be already created in validate/Adults, validate/Teenagers, validate/Toddlers")

# Step 3: Label Encoding
lst = ['Adults', 'Teenagers', 'Toddler']
LE = LabelEncoder()
LE.fit(lst)
print("Label Encoder classes:", LE.classes_)



Validation set assumed to be already created in validate/Adults, validate/Teenagers, validate/Toddlers
Label Encoder classes: ['Adults' 'Teenagers' 'Toddler']


In [ ]:
# Step 5: Initialize ImageDataGenerators with enhanced augmentation
data_gen_train = ImageDataGenerator(
    rotation_range=30,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    brightness_range=[0.8, 1.2],
    fill_mode='nearest',
    preprocessing_function=preprocess_input
)
data_gen_valid = ImageDataGenerator(preprocessing_function=preprocess_input)
data_gen_test = ImageDataGenerator(preprocessing_function=preprocess_input)

# Create generators
batch_size = 16
train_generator = data_gen_train.flow_from_directory(
    train_path,
    target_size=(224, 224),
    class_mode='categorical',
    batch_size=batch_size,
    seed=42
)
validation_generator = data_gen_valid.flow_from_directory(
    validate_path,
    target_size=(224, 224),
    class_mode='categorical',
    batch_size=batch_size,
    seed=42
)
test_generator = data_gen_test.flow_from_directory(
    test_path,
    target_size=(224, 224),
    class_mode=None,
    shuffle=False,
    batch_size=1  # Predict one image at a time
)

# Calculate steps per epoch
steps_per_epoch = train_generator.samples // batch_size
if train_generator.samples % batch_size:
    steps_per_epoch += 1
validation_steps = validation_generator.samples // batch_size
if validation_generator.samples % batch_size:
    validation_steps += 1

print(f"Steps per epoch: {steps_per_epoch}, Validation steps: {validation_steps}")

Found 2813 images belonging to 3 classes.
Found 560 images belonging to 3 classes.
Found 324 images belonging to 1 classes.
Steps per epoch: 176, Validation steps: 35


In [ ]:

# Step 6: Initialize ResNet50V2 base model
img_shape = (224, 224, 3)
fine_tune_at = 170  # Fine-tune more layers
base_model = ResNet50V2(include_top=False, weights='imagenet', input_shape=img_shape)
base_model.trainable = True
for layer in base_model.layers[:fine_tune_at]:
    layer.trainable = False

# Step 7: Add custom layers with increased capacity and regularization
global_average_layer = GlobalAveragePooling2D()(base_model.output)
lay_1 = Dense(units=128, activation='relu', kernel_regularizer=regularizers.l2(0.01))(global_average_layer)
d1 = Dropout(0.3)(lay_1)
lay_2 = Dense(units=64, activation='relu', kernel_regularizer=regularizers.l2(0.01))(d1)
d2 = Dropout(0.3)(lay_2)
prediction_layer = Dense(units=3, activation='softmax')(d2)
model = Model(inputs=base_model.input, outputs=prediction_layer)

# Compile model
model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=0.0001),
    loss='categorical_crossentropy',
    metrics=['accuracy']
)

94668760/94668760 ━━━━━━━━━━━━━━━━━━━━ 3s 0us/step


In [ ]:

# Step 8: Train model with early stopping and learning rate reduction
early_stopping = EarlyStopping(
    monitor='val_loss',
    patience=5,  # Increased patience
    restore_best_weights=True
)
lr_scheduler = ReduceLROnPlateau(
    monitor='val_loss',
    factor=0.5,
    patience=3,
    min_lr=1e-6,
    verbose=1
)

history = model.fit(
    train_generator,
    epochs=20,
    steps_per_epoch=steps_per_epoch,
    validation_data=validation_generator,
    validation_steps=validation_steps,
    callbacks=[early_stopping, lr_scheduler]
)


/usr/local/lib/python3.11/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


Epoch 1/20
176/176 ━━━━━━━━━━━━━━━━━━━━ 1373s 8s/step - accuracy: 0.4235 - loss: 4.2034 - val_accuracy: 0.7607 - val_loss: 3.2786 - learning_rate: 1.0000e-04
Epoch 2/20
176/176 ━━━━━━━━━━━━━━━━━━━━ 839s 5s/step - accuracy: 0.5823 - loss: 3.3891 - val_accuracy: 0.7839 - val_loss: 2.7834 - learning_rate: 1.0000e-04
Epoch 3/20
176/176 ━━━━━━━━━━━━━━━━━━━━ 806s 5s/step - accuracy: 0.6738 - loss: 2.9168 - val_accuracy: 0.8339 - val_loss: 2.3986 - learning_rate: 1.0000e-04
Epoch 4/20
146/176 ━━━━━━━━━━━━━━━━━━━━ 1:52 4s/step - accuracy: 0.7075 - loss: 2.5755

In [ ]:

# Step 9: Plot training history
plt.figure(figsize=(12, 4))
plt.subplot(1, 2, 1)
plt.plot(history.history['accuracy'], label='Train Accuracy')
plt.plot(history.history['val_accuracy'], label='Validation Accuracy')
plt.title('Model Accuracy')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.legend()

plt.subplot(1, 2, 2)
plt.plot(history.history['loss'], label='Train Loss')
plt.plot(history.history['val_loss'], label='Validation Loss')
plt.title('Model Loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.legend()
plt.tight_layout()
plt.savefig('training_history.png')
plt.show()

In [ ]:
# Step 10: Make predictions on test set
predictions = model.predict(test_generator)
predicted_classes = np.argmax(predictions, axis=1)
filenames = [os.path.basename(f) for f in test_generator.filenames]

# Map indices to category names using LabelEncoder
predicted_categories = LE.inverse_transform(predicted_classes)

# Store predictions in a dictionary
match = {filename: category for filename, category in zip(filenames, predicted_categories)}

# Print predictions
print("\nTest Predictions:")
for filename, category in match.items():
    print(f"{filename}: {category}")

# Return predictions for further use if needed
# match is a dictionary with filename as key and predicted category as value